In [1]:
import os

In [2]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir:Path
    data_path:Path

In [6]:
from CreditCardFraud.constants import *
from CreditCardFraud.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml (config_filepath)
        self.params = read_yaml (params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) ->  DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config


In [8]:
import os
from CreditCardFraud.logging import logger
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import joblib

In [9]:
class DataTransformation:
    def __init__(self, config = DataTransformationConfig):
        self.config = config
    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        train , test = train_test_split(data, random_state=0)
        return train , test
    
    def transforming(self,train,test):
        target = 'default payment next month'
        X_train = train.drop(target,axis=1)
        y_train = train[target]
        X_test = test.drop(target,axis=1)
        y_test = test[target]

        numerical_cols = X_train.columns
        
        numerical_pipeline = Pipeline(
                steps=[
                    ('imputer',SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler())
                ]
            )
        
        preprocessor = ColumnTransformer([
                ('numerical_pipeline',numerical_pipeline , numerical_cols)
            ])
        
        X_train_arr = preprocessor.fit_transform(X_train)
        X_test_arr = preprocessor.transform(X_test)

        train_data = pd.DataFrame(np.c_[X_train_arr,y_train],columns=train.columns)
        test_data = pd.DataFrame(np.c_[X_test_arr,y_test],columns=test.columns)
    
        train_data.to_csv(os.path.join(self.config.root_dir , "train.csv"),index=False)
        test_data.to_csv(os.path.join(self.config.root_dir, "test.csv"),index=False)    



        logger.info("Data Transformation Completed")

        logger.info(f"""The shape of the training data is {train_data.shape},
                    the shape of the testing data is {test_data.shape}""")
        preprocessor_path = Path(os.path.join(self.config.root_dir, "preprocessor.joblib"))
        joblib.dump(value=preprocessor ,filename = preprocessor_path)
        logger.info(f"The preprocessor was saved at {preprocessor_path}")


In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    train , test = data_transformation.train_test_splitting()
    data_transformation.transforming(train , test)
except Exception as e:
    raise e

[2023-08-13 00:35:54,334: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-13 00:35:54,334: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-13 00:35:54,334: INFO: common: creating directory at: artifacts]
[2023-08-13 00:35:54,344: INFO: common: creating directory at: artifacts/data_transformation]
[2023-08-13 00:35:54,441: INFO: 250325625: Data Transformation Completed]
[2023-08-13 00:35:54,441: INFO: 250325625: The shape of the training data is (750, 24),
                    the shape of the testing data is (251, 24)]
[2023-08-13 00:35:54,456: INFO: 250325625: The preprocessor was saved at artifacts\data_transformation\preprocessor.joblib]
